In [9]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from numpy import savetxt

In [14]:
class particle:
    """class that define a particle"""
    def __init__(self, n_atom, rads_atom,size = 10,size_grid=32, size_image=25):
        self.n_atom = n_atom
        self.rads_atom = rads_atom
        self.size = size
        self.crds = size * np.random.rand(n_atom, 3)
        self.crds -= np.mean(self.crds, axis=0)
        self.rads = rads_atom
        self.image = None
        self.size_grid = size_grid
        self.size_image = size_image
        self.map = None
        self.last_rot = None
        
    def rotate(self, random = True, quat = None):
        """
        Rotating the particle with the quaternion notation
        """
        if random:
            rotation =  R.random()
        else:
            rotation = R.from_quat(quat)
            
        self.last_rot = rotation.as_rotvec()
        
        for i in range(self.n_atom):
            self.crds[i] = rotation.apply(self.crds[i])
            
        self.create_image()
    
    def create_image(self):
        """
        Return the projection of the particle along the z axis
        """
        image = np.zeros((self.size_grid,self.size_grid))
        for i in range(self.size_grid):
            posx = -self.size_image/2 + i*self.size_image/(self.size_grid-1)
            for j in range(self.size_grid):
                posy = -self.size_image/2 + j*self.size_image/(self.size_grid-1)
                
                for iat in range(self.n_atom):
                    dx = self.crds[iat,0] - posx
                    dy = self.crds[iat,1] - posy
                    dist2 = dx*dx + dy*dy
                    if dist2 <= 100:
                        #integrating along z axis
                        image[i,j] += np.sqrt(np.pi)*np.exp(-0.5*dist2/self.rads[iat]**2)
                        
        self.image = image
    
    def show(self):
        if self.has_image:
            plt.imshow(self.image, cmap='Greys') 
            
    def create_map(self):
        """
        return a 3d array of the particle
        """
        volume = np.zeros((self.size_grid,self.size_grid,self.size_grid))
        for i in range(self.size_grid):
            posx = -self.size_image/2 + i*self.size_image/(self.size_grid-1)
            for j in range(self.size_grid):
                posy = -self.size_image/2 + j*self.size_image/(self.size_grid-1)
                for k in range(self.size_grid):
                    posz = -self.size_image/2 + k*self.size_image/(self.size_grid-1)
                    
                    for iat in range(self.n_atom):
                        dx = self.crds[iat,0] - posx
                        dy = self.crds[iat,1] - posy
                        dz = self.crds[iat,2] - posz
                        dist2  = dx*dx + dy*dy + dz*dz
                        if(dist2 <= 100):
                            volume[i,j,k] += np.exp(-0.5*dist2/self.rads[iat]**2)
        self.volume = volume

In [19]:
def generate_dataset(particle, length, with_orientation = False):
    
    dataset = np.zeros((length, particle.size_grid, particle.size_grid))
    if (with_orientation):
        orientations = np.zeros((length, 3))
        for i in range(length):
            particle.rotate()
            dataset[i] = particle.image
            orientations[i] = particle.last_rot
            if (i+1)%100 == 0:
                print(str(i+1) + ' images were created')
    else:
        for i in range(length):
            particle.rotate()
            dataset[i] = particle.image
            if (i+1)%100 == 0:
                print(str(i+1) + ' images were created')

        
    # save to csv file
    data_name = 'dataset ' + str(particle.n_atom) + ' atoms (' + str(length) + ' images)'
    np.save(data_name, dataset)
    
    particle.create_map()
    map_name = 'reconstructed map of ' + data_name
    np.save(map_name, particle.volume)
    
    if (with_orientation):
        orientations_name = 'orientations of ' + data_name
        np.save(orientations_name, orientations)
    
    

In [12]:
#test
particle_ = particle(3, [1,2,1])
generate_dataset(particle_,1000)

100 images were created
200 images were created
300 images were created
400 images were created
500 images were created
600 images were created
700 images were created
800 images were created
900 images were created
1000 images were created


In [4]:
#higher quality dataset
particle_ = particle(2, [1,2], size_grid = 128)
generate_dataset(particle_,100)

100 images were created


In [20]:
#dataset with orientations
particle_ = particle(3, [1,2,2])
generate_dataset(particle_,200, with_orientation = True)

100 images were created
200 images were created
